In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
# from fastai.callbacks import *

import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.base_ae import BaseAutoEncoder
from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions, DistTypes
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config

from dsin.train_utils.visibilty_utils.logger_cb_utils import setup_file_logger
from dsin.train_utils.visibilty_utils.pbar_cb_utils import (AverageMetric,
                                            ParameterMetricCallback,
                                            ProgressBarAttibuteVisualizer,
                                            ParameterRunningAverageMetricCallback,
                                            BitEntropy
                                           )
from dsin.train_utils.save_utils import FileNameGenerator

Matplotlib is building the font cache using fc-list. This may take a moment.


In [3]:
bpp = 0.1
H_target = 2* bpp
# config.autoencoder_loss_distortion_to_minimize=DistTypes.MS_SSMIM
config.K_MS_SSIM=500
config.beta

500

In [4]:
logger = setup_file_logger()

In [5]:
config.use_si_flag = SiNetChannelIn.WithSideInformation

In [6]:
learn = None
gc.collect()

123

In [7]:
baseline_autoencoder = BaseAutoEncoder()
path = "../src/dsin/data"
pct= 1 # 0.0005 #0.25

valid_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_baseline_val.txt"],pct=pct/10)
train_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_baseline_train.txt"],pct=pct)

image_lists = ItemLists(
    path=path, train=train_image_list, valid=valid_image_list)


tfms =  get_transforms(do_flip=True,
                       flip_vert=False,
                       max_rotate=False,
                       max_zoom=0,
                       max_lighting=False,
                       max_warp=False)
batchsize = 32

data = (image_lists
        .label_from_func(lambda x: x)
        .transform(tfms, size=(192, 192), resize_method=ResizeMethod.CROP, tfm_y=True)
        .databunch(bs=batchsize))
learn = None
gc.collect()
learn = Learner(data=data,
                     model=baseline_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(baseline_autoencoder,SiNetChannelIn.NoSideInformation,target_bit_cost = H_target),
                     metrics=[AverageMetric(Distortions._calc_dist,"MS_SSIM")])

In [11]:
model_fname =FileNameGenerator.create_fname(f'MAE-baseline-bpp0p{bpp*100:.0f}')
lrs = [1e-4] *3 +  [1e-4] *3 + [1e-5] * 4 + [1e-6] *2 
model_fname

'2020-09-06MAE-baseline-bpp0p10'

In [ ]:
config.si_loss_weight_alpha = 0
learn.model.use_side_infomation = SiNetChannelIn.NoSideInformation
learn.loss_func.use_side_infomation = SiNetChannelIn.NoSideInformation
    
offset = 3
if offset != 0 :
    learn.load(f'{model_fname}-{offset - 1}',with_opt=True)
for i,lr in zip(range(offset,len(lrs)),lrs[offset:]):
        
    learn.fit(1, lr=lr,wd=0,callbacks=[ParameterRunningAverageMetricCallback(learn.loss_func,use_si=False),BitEntropy(loss_man=learn.loss_func,logger=logger,use_si=False)])
    
    learn.save(f'{model_fname}-{i}')
    if i % 3 == 0 or i == len(lrs) - 1:
        !aws s3 cp ~/tDSIN/src/dsin/data/models/{model_fname}-{i}.pth  s3://dsin-us/models/
        !aws s3 cp ~/tDSIN/tutorials/out.log s3://dsin-us/models/{model_fname}.log


epoch,train_loss,valid_loss,MS_SSIM,time
0,129.567017,123.045540,0.386872,09:21


iter 0: bpp = 0.022 imp-mean-var(0.02 0.02) total loss156.3  l2reg_loss=14.9autoencoder_loss_value=22.9si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.188 imp-mean-var(0.02 0.02) total loss130.5  l2reg_loss=14.6autoencoder_loss_value=25.3si_loss=0.0feat_loss_value=0.0
iter 1000: bpp = 0.189 imp-mean-var(0.04 0.04) total loss132.2  l2reg_loss=14.4autoencoder_loss_value=24.7si_loss=0.0feat_loss_value=0.0
upload: ../src/dsin/data/models/2020-09-06MAE-baseline-bpp0p10-3.pth to s3://dsin-us/models/2020-09-06MAE-baseline-bpp0p10-3.pth
upload: ./out.log to s3://dsin-us/models/2020-09-06MAE-baseline-bpp0p10.log


epoch,train_loss,valid_loss,MS_SSIM,time
0,78.080490,67.107529,0.371835,09:21


iter 0: bpp = 0.018 imp-mean-var(0.03 0.03) total loss120.6  l2reg_loss=14.4autoencoder_loss_value=24.4si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.151 imp-mean-var(0.02 0.02) total loss87.7  l2reg_loss=14.2autoencoder_loss_value=23.7si_loss=0.0feat_loss_value=0.0
iter 1000: bpp = 0.138 imp-mean-var(0.02 0.02) total loss71.4  l2reg_loss=14.0autoencoder_loss_value=23.7si_loss=0.0feat_loss_value=0.0


epoch,train_loss,valid_loss,MS_SSIM,time
0,77.635536,70.098679,0.337418,09:13


iter 0: bpp = 0.013 imp-mean-var(0.02 0.02) total loss67.0  l2reg_loss=14.0autoencoder_loss_value=24.0si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.145 imp-mean-var(0.03 0.03) total loss77.8  l2reg_loss=14.0autoencoder_loss_value=27.7si_loss=0.0feat_loss_value=0.0
iter 1000: bpp = 0.138 imp-mean-var(0.03 0.03) total loss79.8  l2reg_loss=13.8autoencoder_loss_value=22.5si_loss=0.0feat_loss_value=0.0


epoch,train_loss,valid_loss,MS_SSIM,time


iter 0: bpp = 0.014 imp-mean-var(0.03 0.03) total loss75.6  l2reg_loss=13.8autoencoder_loss_value=21.0si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.131 imp-mean-var(0.03 0.03) total loss66.6  l2reg_loss=13.8autoencoder_loss_value=21.5si_loss=0.0feat_loss_value=0.0


In [ ]:
learn.save(f'{model_fname}-{i}')

!aws s3 cp ~/tDSIN/src/dsin/data/models/{model_fname}-{i}.pth  s3://dsin-us/models/
!aws s3 cp ~/tDSIN/tutorials/out.log s3://dsin-us/models/{model_fname}.log


In [ ]:
mt = learn.model.my_tuple[-3][0,...].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
show_image(Image(mt),figsize=(5,5))
# # print(mn)

In [ ]:
mt = learn.model.my_tuple[3][0,...].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
show_image(Image(mt),figsize=(5,5))